In [0]:
spark


SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:

pip install google-cloud-automl google-cloud-storage google-cloud-bigquery==2.3.1 google-cloud-texttospeech google-cloud-speech


Python interpreter will be restarted.
INFO: pip is looking at multiple versions of cachetools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of cachetools to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. If you want to abort this run, you can press Ctrl + C to do so. To improve how pip performs, tell us what happened here: https://pip.pypa.io/surveys/backtracking
INFO: pip is looking at multiple versions of google-auth to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of google-auth to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the depen

In [0]:
# import library
from google.cloud import storage
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import google.cloud.bigquery
from google.oauth2 import service_account
import functools
import datetime

In [0]:
# GCP project variable

# Project Name
project_id = 'sakshimukkirwar-hw3-proj'

# BigQuery dataset
project_dataset = 'dataset_1'

# BigQuery table name
BQ_table = 'covid_19_geographic_receive'

# GCP Authentication file
BQ_Credentials = '/FileStore/tables/sakshimukkirwar_hw3_proj_17cc082fd25d.json'

# Copy authentication file into Databrick environment at temp location
src_path = BQ_Credentials  # This takes the path from the variable you set earlier
dst_path = "file:///tmp/sakshmukkirwar_hw3_proj_17cc082fd25d.json"  # Destination on the local file system
dbutils.fs.cp(src_path, dst_path)

#Set local variables for credentials
service_account_json = "/tmp/sakshmukkirwar_hw3_proj_17cc082fd25d.json"


In [0]:
# Configure Spark to use the Google Cloud Auth
spark.conf.set("google.cloud.auth.service.account.enable", "true")
spark.conf.set("google.cloud.auth.service.account.json.keyfile", service_account_json)

try:
    # Read data from BigQuery using Spark
    df = spark.read.format('bigquery') \
        .option("credentialsFile", service_account_json) \
        .option("parentProject", project_id) \
        .option("project", project_id) \
        .option("dataset", project_dataset) \
        .load(BQ_table)
    print('Data loaded successfully')
except Exception as e:
    print(e)

# Display the DataFrame
df.show()


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1942500154299085>:3
      1 from pyspark.sql import SparkSession# Configure Spark to use the Google Cloud Auth
      2 spark.conf.set("google.cloud.auth.service.account.enable", "true")
----> 3 spark.conf.set("google.cloud.auth.service.account.json.keyfile", service_account_json)
      5 try:
      6     # Read data from BigQuery using Spark
      7     df = spark.read.format('bigquery') \
      8         .option("credentialsFile", service_account_json) \
      9         .option("parentProject", project_id) \
     10         .option("project", project_id) \
     11         .option("dataset", project_dataset) \
     12         .load(BQ_table)

NameError: name 'service_account_json' is not defined

In [0]:
#PySpark Calculations

In [0]:
#1. Summary Statistics:
df.describe().show()

+-------+------------------+------------------+------------------+---------------------+------------------+------------------+------------------+-------------------------+------+----------------------+--------------------+
|summary|               day|             month|              year|daily_confirmed_cases|      daily_deaths|   confirmed_cases|            deaths|countries_and_territories|geo_id|country_territory_code|       pop_data_2019|
+-------+------------------+------------------+------------------+---------------------+------------------+------------------+------------------+-------------------------+------+----------------------+--------------------+
|  count|             61900|             61900|             61900|                61900|             61900|             61900|             61900|                    61900| 61900|                 61777|               61777|
|   mean|15.628933764135702| 7.067156704361874|2019.9989176090469|    1155.147237479806| 26.05546042003231|1

In [0]:
#2. Number of cases by country:
from pyspark.sql import functions as F

df.groupBy("countries_and_territories").agg(
    F.sum("daily_confirmed_cases").alias("Total Confirmed Cases")
).orderBy(F.desc("Total Confirmed Cases")).show()


+-------------------------+---------------------+
|countries_and_territories|Total Confirmed Cases|
+-------------------------+---------------------+
|     United_States_of_...|             16256754|
|                    India|              9884100|
|                   Brazil|              6901952|
|                   Russia|              2653928|
|                   France|              2376852|
|           United_Kingdom|              1849403|
|                    Italy|              1843712|
|                    Spain|              1730575|
|                Argentina|              1498160|
|                 Colombia|              1425774|
|                  Germany|              1337078|
|                   Mexico|              1250044|
|                   Poland|              1135676|
|                     Iran|              1108269|
|                   Turkey|               995471|
|                     Peru|               984973|
|                  Ukraine|               900666|


In [0]:
#3. Filtering data: Let's say we want to filter out countries with more than 100,000 confirmed cases.
filtered_data = df.groupBy("countries_and_territories").agg(
    F.sum("daily_confirmed_cases").alias("Total Confirmed Cases")
).filter("`Total Confirmed Cases` > 100000")

filtered_data.show()


+-------------------------+---------------------+
|countries_and_territories|Total Confirmed Cases|
+-------------------------+---------------------+
|                   Russia|              2653928|
|                   Sweden|               320098|
|              Philippines|               449400|
|             South_Africa|               860964|
|              Puerto_Rico|               107158|
|                   Turkey|               995471|
|                     Iraq|               574634|
|                  Germany|              1337078|
|                   Jordan|               259614|
|                   France|              2376852|
|                   Greece|               124534|
|                 Slovakia|               132984|
|                Argentina|              1498160|
|                  Belgium|               608001|
|                  Ecuador|               202110|
|                    Qatar|               140961|
|                  Myanmar|               108342|


In [0]:
#4. Deaths vs Confirmed Cases by country:
df.groupBy("countries_and_territories").agg(
    F.sum("daily_confirmed_cases").alias("Total Confirmed Cases"),
    F.sum("daily_deaths").alias("Total Deaths")
).orderBy(F.desc("Total Confirmed Cases")).show()


+-------------------------+---------------------+------------+
|countries_and_territories|Total Confirmed Cases|Total Deaths|
+-------------------------+---------------------+------------+
|     United_States_of_...|             16256754|      299177|
|                    India|              9884100|      143355|
|                   Brazil|              6901952|      181402|
|                   Russia|              2653928|       46941|
|                   France|              2376852|       57911|
|           United_Kingdom|              1849403|       64170|
|                    Italy|              1843712|       64520|
|                    Spain|              1730575|       47624|
|                Argentina|              1498160|       40766|
|                 Colombia|              1425774|       39053|
|                  Germany|              1337078|       21975|
|                   Mexico|              1250044|      113953|
|                   Poland|              1135676|      

In [0]:
#5 Mortality Rate by Country: To determine the mortality rate, we'll calculate the ratio of the total number of deaths to the total number of confirmed cases for each country and then multiply by 100 to get the percentage.

mortality_rate_df = df.groupBy("countries_and_territories").agg(
    F.sum("daily_confirmed_cases").alias("Total Confirmed Cases"),
    F.sum("daily_deaths").alias("Total Deaths")
).withColumn(
    "Mortality Rate (%)",
    (F.col("Total Deaths") / F.col("Total Confirmed Cases") * 100)
).orderBy(F.desc("Mortality Rate (%)"))

mortality_rate_df.show()



+-------------------------+---------------------+------------+------------------+
|countries_and_territories|Total Confirmed Cases|Total Deaths|Mortality Rate (%)|
+-------------------------+---------------------+------------+------------------+
|                    Yemen|                 2083|         606|29.092654824771962|
|                   Mexico|              1250044|      113953| 9.115919119646989|
|               Montserrat|                   13|           1|7.6923076923076925|
|                  Ecuador|               202110|       13875|6.8650734748404325|
|              Isle_of_Man|                  370|          25| 6.756756756756757|
|                    Sudan|                21386|        1347| 6.298513045917891|
|                  Bolivia|               147150|        9018| 6.128440366972477|
|                     Chad|                 1770|         102| 5.762711864406779|
|                    Egypt|               121575|        6920| 5.691959695661115|
|               

In [0]:
# Configure Spark to use the Google Cloud Auth
spark.conf.set("google.cloud.auth.service.enable", "true")
spark.conf.set("google.cloud.auth.service.account.json.keyfile", "/FileStore/tables/sakshimukkirwar_hw3_proj_17cc082fd25d.json")

# Create a DataFrame (replace this with your actual data loading logic)
df = spark.read.format('bigquery') \
    .option("table", "sakshimukkirwar-hw3-proj.dataset_1.covid19_destination_table") \
    .load()

destination_table = "sakshimukkirwar-hw3-proj.dataset_1.covid19_destination_table"

# Write the DataFrame to BigQuery
df.write.format("bigquery") \
    .option("table", destination_table) \
    .mode("overwrite") \
    .save()